### Import packages

In [1]:
from urllib import request
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

# Overall script for downloading data

In [3]:
#Information collected
columnsx=['for','imod','hverkeneller','fravaer','total','samling','lovid','party','name','url','unqiueid','metadata_number','metadata_title']
overall_data=[]
party_data=[]
member_data=pd.DataFrame([])

#Which section of the laws - some of the underlying pages are devided into two sections
collect=['20181','20171','20161','20151','20142','20141','20131','20121','20111','20101','20102','20091','20081','20071','20072','20061','20051','20041','20042']
series_numberoflaws=[454,505,509,595,14,406,414,538,412,468,1,494,488,6,438,536,487,336,305]

i=0

#loop through the different years
for samling in collect: 
    
    numberoflaws=series_numberoflaws[i]
    i = i+1
    #Specify specify text
    for a in range(numberoflaws):
        a +=1
        number=str(a)
        temp=str(samling)
        
        unqiueid=int(samling)*10000000+int(number)
        #------------------------------------------------------------        
        #------------------------------------------------------------        
        # Get the relevant page 
        #------------------------------------------------------------
        #------------------------------------------------------------
        url='https://www.ft.dk/samling/'+temp+'/afstemning/'+number+'.htm'
        print(url)
        response = request.urlopen(url)
        raw = response.read().decode('utf8')
        
        soup = BeautifulSoup(raw, 'lxml')
        text = soup.get_text()

        #------------------------------------------------------------        
        #------------------------------------------------------------        
        # taking out subsection of the text
        #------------------------------------------------------------
        #------------------------------------------------------------
                    
        index_start_overall=text.index("Samlet afstemningsresultat")
        index_end_overall=text.index("Afstemningsresultat fordelt på folketingsgrupper")

        index_start_party=text.index("Afstemningsresultat fordelt på folketingsgrupper")
        index_end_party=text.index("Afstemningsresultat fordelt på medlemmer")

        index_start_member=text.index("Afstemningsresultat fordelt på medlemmer")
        index_end_member=text.index("Medlemmer på orlov")

        index_start_extra=text.index("Medlemmer på orlov")
        index_end_extra=text.index("Hvorfor stemmer ikke alle medlemmer")

        text_vote_overall = text[index_start_overall:index_end_overall]
        text_vote_party =text[index_start_party:index_end_party]
        text_vote_member = text[index_start_member:index_end_member]
        text_vote_extra = text[index_start_extra:index_end_extra]
      
        #------------------------------------------------------------        
        #------------------------------------------------------------        
        # Extracting text for metadata
        #------------------------------------------------------------
        #------------------------------------------------------------
        
    
        # handle inconsistency of some vote blocks not having the same info regarding afstemningsnummer - general blank votes e.g
        # https://www.ft.dk/samling/20072/afstemning/1.htm
        # This "samling" only starts from law 3
        try:
            index_start_meta=text.index("Afstemningsnummer") 
            index_end_meta=text.index("Samlet afstemningsresultat")
            text_vote_meta = text[index_start_meta:index_end_meta]
            x=re.sub('  ', '', text_vote_meta)
            z=re.sub('\n', '', x)
            y=re.sub('\r\r', ';', z)
            w=re.sub('\r', ';', y)
            q=re.sub(';;;', '', w)
            r=re.split(';', q)
            metadata_number=r[1]
            metadata_title=r[2]
        except ValueError:
            metadata_number=''
            metadata_title=''
              
        #------------------------------------------------------------        
        #------------------------------------------------------------        
        # Cleaning text for overall votes section 
        #------------------------------------------------------------
        #------------------------------------------------------------        
        
        
        identify_for="For\r\n"
        identify_imod="Imod\r\n"
        identify_hverken="Hverken for/imod\r\n"
        identify_fravaer="Fraværende\r\n"
        textlenght=50

        start=text_vote_overall.index(identify_for)
        end=start+textlenght
        temp=text_vote_overall[start:end]
        vote_overall_for=re.findall('\d+',temp)

        start=text_vote_overall.index(identify_imod)
        end=start+textlenght
        temp=text_vote_overall[start:end]
        vote_overall_imod=re.findall('\d+',temp)

        start=text_vote_overall.index(identify_hverken)
        end=start+textlenght
        temp=text_vote_overall[start:end]
        vote_overall_hverken=re.findall('\d+',temp)

        start=text_vote_overall.index(identify_fravaer)
        end=start+textlenght
        temp=text_vote_overall[start:end]
        vote_overall_fravaer=re.findall('\d+',temp)

        votes_overall=vote_overall_for+vote_overall_imod+vote_overall_hverken+vote_overall_fravaer
        #compute a total column
        temp=0
        party=''
        name=''
        for a in votes_overall:
            z=int(a)
            temp += z
            out=(str(temp))
        votes_overall.append(out)
        votes_overall.append(samling)
        votes_overall.append(number)   
        votes_overall.append(party)
        votes_overall.append(name)  
        votes_overall.append(url)  
        votes_overall.append(unqiueid) 
        votes_overall.append(metadata_number)  
        votes_overall.append(metadata_title) 
        
        #append data
        overall_data.append(votes_overall)
        
        #------------------------------------------------------------        
        #------------------------------------------------------------        
        # Cleaning text for party votes section 
        #------------------------------------------------------------
        #------------------------------------------------------------
    
        blocklength=200
        partytag_str = '\(([A-Z]+)\)'
        partytag_str_re = re.compile(partytag_str, flags = re.DOTALL|re.UNICODE)
        party_index = re.findall(partytag_str_re, text_vote_party)

        for a in party_index:
            partytag = '('+a+')' #search for blocks of text near parties
            start=text_vote_party.index(partytag)
            end=start+blocklength #take out text around start of a new party
            temp=text_vote_party[start:end]
            votes_temp=re.findall('\d+',temp) #find numbers here --> votes in sequence for, imod, hverkeneller, fraværende

            temp=0
            party=a
            name=''
            for a in votes_temp:
                z=int(a)
                temp += z
                out=(str(temp))
            votes_temp.append(out)
            votes_temp.append(samling)
            votes_temp.append(number)   
            votes_temp.append(party)
            votes_temp.append(name)  
            votes_temp.append(url) 
            votes_temp.append(unqiueid) 
            votes_temp.append(metadata_number)  
            votes_temp.append(metadata_title) 
            
            #append data
            party_data.append(votes_temp)
            
        #------------------------------------------------------------        
        #------------------------------------------------------------        
        # Cleaning text for members votes section 
        #------------------------------------------------------------
        #------------------------------------------------------------            
            
        #split out sequence of text
        list=[]
        test=re.split('\n\r',text_vote_member)
        for a in test:
            b=re.sub('\n', '', a)
            c=re.sub('\r', '', b)
            d=re.sub('  ', '', c)
            list.append(d)
            list2=list[4:len(list)]

        #initilialize containers
        count=0
        name=[]
        party=[]
        party_full=[]
        vote=[]
        samling2=[]
        number2=[]
        url2=[]
        unqiueid2=[]

        #cycle through name, party, vote
        for b in list2:
            count= count +1
            samling2.append(samling)
            number2.append(number)
            url2.append(url)
            unqiueid2.append(unqiueid)
            if count==1:
                name.append(b)
            if count==2:
                party_full.append(b)
            if count==3:
                vote.append(b)
                count = 0 #rest to repeat

        #create dataframes
        a=pd.DataFrame(vote, columns=['vote'])
        b=pd.DataFrame(name, columns=['name'])
        c=pd.DataFrame(party_full, columns=['party_full'])
        d=pd.DataFrame(samling2, columns=['samling'])
        e=pd.DataFrame(number2, columns=['number'])
        #f=pd.DataFrame(url2, columns=['url']) #no need to save url in largedataset
        g=pd.DataFrame(unqiueid2, columns=['unqiueid'])

        for parties in c.party_full:
            temp=re.findall('\(([A-Z]+)\)',parties)
            party.append(temp)
        h=pd.DataFrame(party, columns=['party'])

        #concat and constructs vars
        frames = [a, b, c, d, e, g, h]
        temp_member = pd.concat(frames, axis=1, join_axes=[a.index])
        temp_member['for'] = np.where(temp_member['vote']=='For',1,0)
        temp_member['imod'] = np.where(temp_member['vote']=='Imod',1,0)
        temp_member['hverkeneller'] = np.where(temp_member['vote']=='Hverken for eller imod',1,0)
        temp_member['fravaer'] = np.where(temp_member['vote']=='Fraværende',1,0)
        temp_member['total'] = 1

        #append dataset
        member_data=pd.concat([member_data,temp_member],ignore_index=True)
        
        #delay 1 second
        time.sleep(1)
        
df_overall=pd.DataFrame(overall_data,columns=columnsx)
df_party=pd.DataFrame(party_data,columns=columnsx)
df_member=pd.DataFrame(member_data)

#output to csv
df_overall.to_csv('df_overall.csv')
df_party.to_csv('df_party.csv')
df_member.to_csv('df_member.csv')

df_overall.to_pickle('df_overall.pkl')
df_party.to_pickle('df_party.pkl')
df_member.to_pickle('df_member.pkl')

print('done')

https://www.ft.dk/samling/20051/afstemning/1.htm
https://www.ft.dk/samling/20051/afstemning/2.htm
https://www.ft.dk/samling/20051/afstemning/3.htm
https://www.ft.dk/samling/20051/afstemning/4.htm
https://www.ft.dk/samling/20051/afstemning/5.htm
https://www.ft.dk/samling/20051/afstemning/6.htm
https://www.ft.dk/samling/20051/afstemning/7.htm
https://www.ft.dk/samling/20051/afstemning/8.htm
https://www.ft.dk/samling/20051/afstemning/9.htm
https://www.ft.dk/samling/20051/afstemning/10.htm
https://www.ft.dk/samling/20051/afstemning/11.htm
https://www.ft.dk/samling/20051/afstemning/12.htm
https://www.ft.dk/samling/20051/afstemning/13.htm
https://www.ft.dk/samling/20051/afstemning/14.htm
https://www.ft.dk/samling/20051/afstemning/15.htm
https://www.ft.dk/samling/20051/afstemning/16.htm
https://www.ft.dk/samling/20051/afstemning/17.htm
https://www.ft.dk/samling/20051/afstemning/18.htm
https://www.ft.dk/samling/20051/afstemning/19.htm
https://www.ft.dk/samling/20051/afstemning/20.htm
https://w

https://www.ft.dk/samling/20051/afstemning/164.htm
https://www.ft.dk/samling/20051/afstemning/165.htm
https://www.ft.dk/samling/20051/afstemning/166.htm
https://www.ft.dk/samling/20051/afstemning/167.htm
https://www.ft.dk/samling/20051/afstemning/168.htm
https://www.ft.dk/samling/20051/afstemning/169.htm
https://www.ft.dk/samling/20051/afstemning/170.htm
https://www.ft.dk/samling/20051/afstemning/171.htm
https://www.ft.dk/samling/20051/afstemning/172.htm
https://www.ft.dk/samling/20051/afstemning/173.htm
https://www.ft.dk/samling/20051/afstemning/174.htm
https://www.ft.dk/samling/20051/afstemning/175.htm
https://www.ft.dk/samling/20051/afstemning/176.htm
https://www.ft.dk/samling/20051/afstemning/177.htm
https://www.ft.dk/samling/20051/afstemning/178.htm
https://www.ft.dk/samling/20051/afstemning/179.htm
https://www.ft.dk/samling/20051/afstemning/180.htm
https://www.ft.dk/samling/20051/afstemning/181.htm
https://www.ft.dk/samling/20051/afstemning/182.htm
https://www.ft.dk/samling/20051

https://www.ft.dk/samling/20051/afstemning/325.htm
https://www.ft.dk/samling/20051/afstemning/326.htm
https://www.ft.dk/samling/20051/afstemning/327.htm
https://www.ft.dk/samling/20051/afstemning/328.htm
https://www.ft.dk/samling/20051/afstemning/329.htm
https://www.ft.dk/samling/20051/afstemning/330.htm
https://www.ft.dk/samling/20051/afstemning/331.htm
https://www.ft.dk/samling/20051/afstemning/332.htm
https://www.ft.dk/samling/20051/afstemning/333.htm
https://www.ft.dk/samling/20051/afstemning/334.htm
https://www.ft.dk/samling/20051/afstemning/335.htm
https://www.ft.dk/samling/20051/afstemning/336.htm
https://www.ft.dk/samling/20051/afstemning/337.htm
https://www.ft.dk/samling/20051/afstemning/338.htm
https://www.ft.dk/samling/20051/afstemning/339.htm
https://www.ft.dk/samling/20051/afstemning/340.htm
https://www.ft.dk/samling/20051/afstemning/341.htm
https://www.ft.dk/samling/20051/afstemning/342.htm
https://www.ft.dk/samling/20051/afstemning/343.htm
https://www.ft.dk/samling/20051

https://www.ft.dk/samling/20051/afstemning/486.htm
https://www.ft.dk/samling/20051/afstemning/487.htm
https://www.ft.dk/samling/20041/afstemning/1.htm
https://www.ft.dk/samling/20041/afstemning/2.htm
https://www.ft.dk/samling/20041/afstemning/3.htm
https://www.ft.dk/samling/20041/afstemning/4.htm
https://www.ft.dk/samling/20041/afstemning/5.htm
https://www.ft.dk/samling/20041/afstemning/6.htm
https://www.ft.dk/samling/20041/afstemning/7.htm
https://www.ft.dk/samling/20041/afstemning/8.htm
https://www.ft.dk/samling/20041/afstemning/9.htm
https://www.ft.dk/samling/20041/afstemning/10.htm
https://www.ft.dk/samling/20041/afstemning/11.htm
https://www.ft.dk/samling/20041/afstemning/12.htm
https://www.ft.dk/samling/20041/afstemning/13.htm
https://www.ft.dk/samling/20041/afstemning/14.htm
https://www.ft.dk/samling/20041/afstemning/15.htm
https://www.ft.dk/samling/20041/afstemning/16.htm
https://www.ft.dk/samling/20041/afstemning/17.htm
https://www.ft.dk/samling/20041/afstemning/18.htm
https:/

https://www.ft.dk/samling/20041/afstemning/162.htm
https://www.ft.dk/samling/20041/afstemning/163.htm
https://www.ft.dk/samling/20041/afstemning/164.htm
https://www.ft.dk/samling/20041/afstemning/165.htm
https://www.ft.dk/samling/20041/afstemning/166.htm
https://www.ft.dk/samling/20041/afstemning/167.htm
https://www.ft.dk/samling/20041/afstemning/168.htm
https://www.ft.dk/samling/20041/afstemning/169.htm
https://www.ft.dk/samling/20041/afstemning/170.htm
https://www.ft.dk/samling/20041/afstemning/171.htm
https://www.ft.dk/samling/20041/afstemning/172.htm
https://www.ft.dk/samling/20041/afstemning/173.htm
https://www.ft.dk/samling/20041/afstemning/174.htm
https://www.ft.dk/samling/20041/afstemning/175.htm
https://www.ft.dk/samling/20041/afstemning/176.htm
https://www.ft.dk/samling/20041/afstemning/177.htm
https://www.ft.dk/samling/20041/afstemning/178.htm
https://www.ft.dk/samling/20041/afstemning/179.htm
https://www.ft.dk/samling/20041/afstemning/180.htm
https://www.ft.dk/samling/20041

https://www.ft.dk/samling/20041/afstemning/323.htm
https://www.ft.dk/samling/20041/afstemning/324.htm
https://www.ft.dk/samling/20041/afstemning/325.htm
https://www.ft.dk/samling/20041/afstemning/326.htm
https://www.ft.dk/samling/20041/afstemning/327.htm
https://www.ft.dk/samling/20041/afstemning/328.htm
https://www.ft.dk/samling/20041/afstemning/329.htm
https://www.ft.dk/samling/20041/afstemning/330.htm
https://www.ft.dk/samling/20041/afstemning/331.htm
https://www.ft.dk/samling/20041/afstemning/332.htm
https://www.ft.dk/samling/20041/afstemning/333.htm
https://www.ft.dk/samling/20041/afstemning/334.htm
https://www.ft.dk/samling/20041/afstemning/335.htm
https://www.ft.dk/samling/20041/afstemning/336.htm
https://www.ft.dk/samling/20042/afstemning/1.htm
https://www.ft.dk/samling/20042/afstemning/2.htm
https://www.ft.dk/samling/20042/afstemning/3.htm
https://www.ft.dk/samling/20042/afstemning/4.htm
https://www.ft.dk/samling/20042/afstemning/5.htm
https://www.ft.dk/samling/20042/afstemnin

https://www.ft.dk/samling/20042/afstemning/150.htm
https://www.ft.dk/samling/20042/afstemning/151.htm
https://www.ft.dk/samling/20042/afstemning/152.htm
https://www.ft.dk/samling/20042/afstemning/153.htm
https://www.ft.dk/samling/20042/afstemning/154.htm
https://www.ft.dk/samling/20042/afstemning/155.htm
https://www.ft.dk/samling/20042/afstemning/156.htm
https://www.ft.dk/samling/20042/afstemning/157.htm
https://www.ft.dk/samling/20042/afstemning/158.htm
https://www.ft.dk/samling/20042/afstemning/159.htm
https://www.ft.dk/samling/20042/afstemning/160.htm
https://www.ft.dk/samling/20042/afstemning/161.htm
https://www.ft.dk/samling/20042/afstemning/162.htm
https://www.ft.dk/samling/20042/afstemning/163.htm
https://www.ft.dk/samling/20042/afstemning/164.htm
https://www.ft.dk/samling/20042/afstemning/165.htm
https://www.ft.dk/samling/20042/afstemning/166.htm
https://www.ft.dk/samling/20042/afstemning/167.htm
https://www.ft.dk/samling/20042/afstemning/168.htm
https://www.ft.dk/samling/20042

# Data joins 2004 and rest

This step is done, as the scraping was halted in 2004 year and had to be redone

In [ ]:
'''
df_overall20042005=pd.DataFrame(overall_data,columns=columnsx)
df_party20042005=pd.DataFrame(party_data,columns=columnsx)
df_member20042005=pd.DataFrame(member_data)

#output to csv
df_overall20042005.to_csv('df_overall20042005.csv')
df_party20042005.to_csv('df_party20042005.csv')
df_member20042005.to_csv('df_member20042005.csv')

df_overall20042005.to_pickle('df_overall20042005.pkl')
df_party20042005.to_pickle('df_party20042005.pkl')
df_member20042005.to_pickle('df_member20042005.pkl')
'''

In [9]:
'''
#TEMP SAVES: 2005 is partly included --> should be removed before merge
df_overall2018_2005plus=pd.DataFrame(overall_data,columns=columnsx)
df_party2018_2005plus=pd.DataFrame(party_data,columns=columnsx)
df_member2018_2005plus=pd.DataFrame(member_data)

#output to csv
df_overall2018_2005plus.to_csv('df_overall2018_2005plus.csv')
df_party2018_2005plus.to_csv('df_party2018_2005plus.csv')
df_member2018_2005plus.to_csv('df_member2018_2005plus.csv')

df_overall2018_2005plus.to_pickle('df_overall2018_2005plus.pkl')
df_party2018_2005plus.to_pickle('df_party2018_2005plus.pkl')
df_member2018_2005plus.to_pickle('df_member2018_2005plus.pkl')
'''

In [83]:
'''
#overall
df_overall2018_2005plus=pd.read_csv('df_overall2018_2005plus.csv')
df_overall2018_2005plus = df_overall2018_2005plus[(df_overall2018_2005plus.samling !=20051) & (df_overall2018_2005plus.samling !=20052)]
df_overall20042005=pd.read_csv('df_overall20042005.csv')

frames = [df_overall2018_2005plus, df_overall20042005]
df_overall = pd.concat(frames)
df_overall.reset_index()
df_overall.to_csv('df_overall.csv')
df_overall.to_pickle('df_overall.pkl')

#party
df_party2018_2005plus=pd.read_csv('df_party2018_2005plus.csv')
df_party2018_2005plus = df_party2018_2005plus[(df_party2018_2005plus.samling !=20051) & (df_party2018_2005plus.samling !=20052)]
df_party20042005=pd.read_csv('df_party20042005.csv')

frames = [df_party2018_2005plus, df_party20042005]
df_party = pd.concat(frames)
df_party.reset_index()
df_party.to_csv('df_party.csv')
df_party.to_pickle('df_party.pkl')

#member
#Note to self: if scraped again; sampling has been corrected to samling
df_member2018_2005plus=pd.read_csv('df_member2018_2005plus.csv') 
df_member2018_2005plus = df_member2018_2005plus[(df_member2018_2005plus.samling !=20051) & (df_member2018_2005plus.samling !=20052)]
df_member20042005=pd.read_csv('df_member20042005.csv')

frames = [df_member2018_2005plus, df_member20042005]
df_member = pd.concat(frames)
df_member.reset_index()
df_member.to_csv('df_member.csv')
df_member.to_pickle('df_member.pkl')
'''

## Data checks

In [251]:
df_overall=pd.read_csv('df_overall.csv')
df_member=pd.read_csv('df_member.csv')
df_party=pd.read_csv('df_party.csv')

In [252]:
b=0
for a in series_numberoflaws:
    b+=a
print('Total number of laws itterated over')
print(b)

Total number of laws itterated over
7406


In [253]:
len(df_overall.groupby(['unqiueid']))

7406

In [257]:
df_overall['zerovotes'] = np.where(df_overall['total']==0,1,0)
df_overall2=df_overall[df_overall['zerovotes']==0]
laws_to_be_removed=df_overall[df_overall['zerovotes']==1]
laws_to_be_removed=laws_to_be_removed[['unqiueid','zerovotes','url']]

In [258]:
len(laws_to_be_removed.groupby(['unqiueid']))
# These appear legit

72

In [260]:
laws_to_be_removed.url.head()

588     https://www.ft.dk/samling/20171/afstemning/135...
1461    https://www.ft.dk/samling/20161/afstemning/503...
1462    https://www.ft.dk/samling/20161/afstemning/504...
1463    https://www.ft.dk/samling/20161/afstemning/505...
1464    https://www.ft.dk/samling/20161/afstemning/506...
Name: url, dtype: object

# 7334 laws has been scraped succesfully

In [262]:
#These appear legit, also
df_overall['tally'] = df_overall['for']+df_overall['imod']+df_overall['hverkeneller']+df_overall['fravaer']
df_overall['zerovotes'] = np.where(df_overall['total']==0,1,0)
df_overall2=df_overall[df_overall['zerovotes']==0]
laws_to_be_removed=df_overall[df_overall['zerovotes']==1]
laws_to_be_removed=laws_to_be_removed[['unqiueid','zerovotes','url']]

df_overall2[df_overall2.tally<179][['for','imod','hverkeneller','fravaer','tally','total']].head()

,for,imod,hverkeneller,fravaer,tally,total
3093,72,38,0,68,178,178
3094,58,52,0,68,178,178
3095,13,97,0,68,178,178
3096,99,10,0,69,178,178
3097,108,0,0,70,178,178


In [264]:
check_overall=df_overall2.tally.value_counts()
check_overall

179    7143
180     141
178      28
181      14
176       4
182       1
177       1
144       1
104       1
Name: tally, dtype: int64

In [265]:
df_overall2[df_overall2.tally<179].url.head()

3093    https://www.ft.dk/samling/20121/afstemning/197...
3094    https://www.ft.dk/samling/20121/afstemning/198...
3095    https://www.ft.dk/samling/20121/afstemning/199...
3096    https://www.ft.dk/samling/20121/afstemning/200...
3097    https://www.ft.dk/samling/20121/afstemning/201...
Name: url, dtype: object

In [266]:
#These appear legit, also
df_overall2[df_overall2.tally>179][['for','imod','hverkeneller','fravaer','tally','total']].head()

,for,imod,hverkeneller,fravaer,tally,total
962,88,22,0,70,180,180
963,82,0,23,75,180,180
964,106,0,0,74,180,180
965,54,53,0,73,180,180
966,54,55,0,71,180,180


In [267]:
df_overall2[df_overall2.tally>179].url.head()

962    https://www.ft.dk/samling/20161/afstemning/4.htm
963    https://www.ft.dk/samling/20161/afstemning/5.htm
964    https://www.ft.dk/samling/20161/afstemning/6.htm
965    https://www.ft.dk/samling/20161/afstemning/7.htm
966    https://www.ft.dk/samling/20161/afstemning/8.htm
Name: url, dtype: object

In [268]:
temp=df_party.groupby(['unqiueid']).sum()
check_party=temp.total.value_counts()
check_party

179    7143
180     141
178      28
181      14
176       4
182       1
177       1
144       1
104       1
Name: total, dtype: int64

In [269]:
temp=df_member.groupby(['unqiueid']).sum()
check_member=temp.total.value_counts()
check_member

179    7143
180     141
178      28
181      14
176       4
182       1
177       1
144       1
104       1
Name: total, dtype: int64

In [270]:
df_member.one=1

In [271]:
temp=df_member.groupby(['unqiueid','samling']).sum()

In [272]:
temp=temp.reset_index()

In [273]:
temp.samling.value_counts()

20151    595
20121    538
20061    535
20171    504
20161    502
20091    493
20051    486
20081    474
20101    468
20181    454
20072    425
20131    414
20111    412
20141    406
20041    304
20042    304
20142     14
20071      5
20102      1
Name: samling, dtype: int64

In [274]:
df_overall2.samling.value_counts()

20151    595
20121    538
20061    535
20171    504
20161    502
20091    493
20051    486
20081    474
20101    468
20181    454
20072    425
20131    414
20111    412
20141    406
20041    304
20042    304
20142     14
20071      5
20102      1
Name: samling, dtype: int64